In [9]:
%reload_ext autoreload
%autoreload 1
import torch 
import sys
sys.path.append('..')
from torch import nn 
from torch.nn import functional as F
from torch import optim
from utils.loader import load
# from models.vince_models import Net2B

In [10]:
# load the dataset
train_input,train_target, train_classes, test_input, test_target, test_classes = load()
train_target=train_target.float() # necessary ?
test_target=test_target.float()

In [11]:
# baseline model
class Net2C(nn.Module):

    def __init__(self, nb_hidden, name = None):
        
        super(Net2C, self).__init__()
        self.name = name
        self.conv1 = nn.Conv2d(2, 32, kernel_size = 5) # number of input channels is 2.
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 3)
        self.fc1 = nn.Linear(256, nb_hidden)
        #self.dropout = nn.Dropout()
        self.fc2 = nn.Linear(nb_hidden, 1) # single output
        self.sigmoid = nn.Sigmoid()  
        
    def forward(self, x):
    
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3, stride = 1))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size = 3, stride = 3))
        x = F.relu(self.fc1(x.view(-1, 256)))
        #x = self.dropout(x)
        x = self.fc2(x)
        x = self.sigmoid(x) 
        return x # returns a probability 

# inspired from LeNet5 but slightly different and modified for binary classification 

class Netcat(nn.Module):      
    
    def __init__(self, name = None):
        
        super(Netcat, self).__init__()
        self.name = name
        self.conv1 = nn.Conv2d(1, 6, kernel_size = 3, stride = 1)
        self.conv2 = nn.Conv2d(6, 16, kernel_size = 3, stride = 1)
        self.fc1 = nn.Linear(480, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)   # add final layer for binary classificaton
        self.sigmoid = nn.Sigmoid()
  
        
    def forward(self, x):
   
        x = torch.cat((x[:,0], x[:,1]), dim = 2).unsqueeze(dim = 1)   # concatenate channels into 1 channel (input : 1000 * 1 * 14 * 28)
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 2, stride = 2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size = 2, stride = 1))
        #print(x.size())
        x = F.relu(self.fc1(x.view(-1, 480)))
        x = F.relu(self.fc2(x.view(-1, 120)))
        x = self.fc3(x)
        x = self.sigmoid(x) 
        return x # returns a probability
        

In [12]:
def train_binary(model, epoch, train_input, train_target, batch_size, 
                 criterion = nn.BCELoss(), optimizer = optim.SGD, 
                 eta = 1e-1, penalty = None, lambda_l2 = 1e-3):  
    # train a binary classifier by minimizing the binary cross entropy loss of a sigmoid output
    # not BCELossWithLogits because sigmoid activation of output already applied in model architecture
    model.train()
    optimizer = optimizer(model.parameters(), lr = eta)
    train_loss = 0
    
    for input_, target in (zip(train_input.split(batch_size), train_target.split(batch_size))):
        output = model(input_)
        batch_loss = criterion(output, target.unsqueeze(1))
        
        if penalty is not None:
            for p in model.parameters():
                batch_loss += lambda_l2 * p.pow(2).sum() # L2 penalty term
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        train_loss += batch_loss   
    print('Train Epoch: {}  | Loss {:.6f}'.format(
                epoch, train_loss.item()))
    
    

def test_binary(model, test_input, test_target, batch_size, criterion = nn.BCELoss()):
    # test a binary classifier using sigmoid followed by rounding ptobability to 0 or 1
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        print('Model Name : {}'.format(model.name))
        
        for data, target in zip(test_input.split(batch_size), test_target.split(batch_size)):
            output = model(data)
            batch_loss = criterion(output, target.unsqueeze(1))
            test_loss += batch_loss
            pred = output 
            correct += torch.round(output).squeeze().eq(target).sum() # only need to round because sigmoid already applied in model 
        print('\nTest set:  Loss: {:.4f}, Accuracy: {:.0f}%\n'.format(
        test_loss.item(), 100 * correct/len(test_target)))

In [14]:
###############################
###### Binary Classifier ######
###############################

# initialize models and normalize data
mean, std = train_input.mean(), train_input.std()
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

model1 = Net2C(100, 'net2c')
model2 = Netcat('netcat')


# train models
nb_epochs = 50

print('Model Name : {}'.format(model1.name))

for epoch in range(1, nb_epochs):
    train_binary(model1, epoch, train_input, train_target, 100)


print('Model Name : {}'.format(model2.name))

for epoch in range(1, nb_epochs):
    train_binary(model2, epoch, train_input, train_target, 100)
    
    
    

Model Name : net2c
Train Epoch: 1  | Loss 6.882156
Train Epoch: 2  | Loss 6.760507
Train Epoch: 3  | Loss 6.654244
Train Epoch: 4  | Loss 6.524962
Train Epoch: 5  | Loss 6.374737
Train Epoch: 6  | Loss 6.157786
Train Epoch: 7  | Loss 5.887795
Train Epoch: 8  | Loss 5.602406
Train Epoch: 9  | Loss 5.328022
Train Epoch: 10  | Loss 5.047230
Train Epoch: 11  | Loss 5.250333
Train Epoch: 12  | Loss 4.810938
Train Epoch: 13  | Loss 4.722609
Train Epoch: 14  | Loss 4.395001
Train Epoch: 15  | Loss 4.259694
Train Epoch: 16  | Loss 4.059671
Train Epoch: 17  | Loss 3.813713
Train Epoch: 18  | Loss 3.672017
Train Epoch: 19  | Loss 3.363561
Train Epoch: 20  | Loss 3.290596
Train Epoch: 21  | Loss 2.700757
Train Epoch: 22  | Loss 3.467858
Train Epoch: 23  | Loss 2.968717
Train Epoch: 24  | Loss 2.131712
Train Epoch: 25  | Loss 3.768393
Train Epoch: 26  | Loss 2.498036
Train Epoch: 27  | Loss 2.113636
Train Epoch: 28  | Loss 1.547755
Train Epoch: 29  | Loss 1.794000
Train Epoch: 30  | Loss 2.170410


In [18]:
# test
test_binary(model1, test_input, test_target, 100)
test_binary(model2, test_input, test_target, 100)

train_classes.size()

Model Name : net2c

Test set:  Loss: 7.6143, Accuracy: 80%

Model Name : netcat

Test set:  Loss: 4.5140, Accuracy: 81%



torch.Size([1000, 2])